# Sampling and homology via bottlenecks

The paper [Sampling and homology via bottlenecks]() by Di Rocco et. al. introduces a novel algorithm for producing a provably dense sampling of a smooth and compact algebraic variety where the density is determined by the smallest bottleneck. Using the sample, the zeroth and first homology of the variety can be recovered using a modified Vietoris-Rips construction. This notebook implements the algorithm for sampling and homology computation for the case of complete intersections and illustrates it using a curve in $\Bbb R^2$.

In [160]:
using HomotopyContinuation, DynamicPolynomials, LinearAlgebra, IterTools

n=5 # ambient dimension
@polyvar x[1:n] y[1:n] p[1:n] gamma[1:n]

F_old = [
    (2*x[1]^2+3*x[2]^2+2*x[3]^2+10^2-5^2)^2 - 4*10^2*(2*x[1]^2+3*x[2]^2)+
    0.00058051 *(1 + 0.335943* x[1] + 0.109178* x[1]^2 + 0.0818345* x[1]^3 + 
   0.404029* x[2] + 0.398644* x[1]* x[2] + 0.0440627* x[1]^2* x[2] + 0.02358* x[2]^2 + 
   0.0398969* x[1]* x[2]^2 + 0.0487113* x[2]^3 + 0.598636* x[3] + 0.0206622* x[1]* x[3] + 
   0.329206* x[1]^2* x[3] + 0.0132711* x[2]* x[3] + 0.0308809* x[1]* x[2] *x[3] + 
   0.000720102* x[2]^2 *x[3] + 0.270658* x[3]^2 + 0.0144158* x[1]* x[3]^2 + 
   0.0618866 *x[2] *x[3]^2 + 0.0484627* x[3]^3)
]
F = [
    x[1]^5+x[2]^5+x[3]^5+x[4]^5+x[5]^5+
    0.0008481796276745865*(1 + 0.3284690640358132*x[1] + 0.1315503107711005*x[1]^2 + 
  0.01980180174132161*x[1]^3 + 0.0018669554790220978*x[1]^4 + 
  0.35737855642712596*x[1]^5 + 0.914346284924554*x[2] + 0.3387230390061304*x[1]*x[2] + 
  0.6319323689224611*x[1]^2*x[2] + 0.05154947362789917*x[1]^3*x[2] + 
  0.09893949213462587*x[1]^4*x[2] + 0.25512397076954685*x[2]^2 + 
  0.03442939179518614*x[1]*x[2]^2 + 0.026240222503881003*x[1]^2*x[2]^2 + 
  0.0702502746030664*x[1]^3*x[2]^2 + 0.07057897442218955*x[2]^3 + 
  0.0005250333579595096*x[1]*x[2]^3 + 0.0006468177239057545*x[1]^2*x[2]^3 + 
  0.12707588344075418*x[2]^4 + 0.10463106719370346*x[1]*x[2]^4 + 
  0.004715612894288556*x[2]^5 + 0.034081890537919435*x[3] + 
  0.004506679286004798*x[1]*x[3] + 0.024012057514701313*x[1]^2*x[3] + 
  0.0029011291069616744*x[1]^3*x[3] + 0.03470169578468609*x[1]^4*x[3] + 
  0.5194880568042196*x[2]*x[3] + 0.09474919819989398*x[1]*x[2]*x[3] + 
  0.08120569160009626*x[1]^2*x[2]*x[3] + 0.02060119519334457*x[1]^3*x[2]*x[3] + 
  0.19459359576033736*x[2]^2*x[3] + 0.0024424337436586498*x[1]*x[2]^2*x[3] + 
  0.2595296163957923*x[1]^2*x[2]^2*x[3] + 0.029345030688392764*x[2]^3*x[3] + 
  0.0002694226953887099*x[1]*x[2]^3*x[3] + 0.0021487064719678543*x[2]^4*x[3] + 
  0.2474144803911428*x[3]^2 + 0.11613496418801923*x[1]*x[3]^2 + 
  0.04868814577952961*x[1]^2*x[3]^2 + 0.008130194230333393*x[1]^3*x[3]^2 + 
  0.04829252724861684*x[2]*x[3]^2 + 0.002669546160260514*x[1]*x[2]*x[3]^2 + 
  0.0006534650431599028*x[1]^2*x[2]*x[3]^2 + 0.039399174792971045*x[2]^2*x[3]^2 + 
  0.0012595209323951208*x[1]*x[2]^2*x[3]^2 + 0.04354704288615972*x[2]^3*x[3]^2 + 
  0.007629554210630074*x[3]^3 + 0.06304627598951223*x[1]*x[3]^3 + 
  0.19994812756333327*x[1]^2*x[3]^3 + 0.012148889622780042*x[2]*x[3]^3 + 
  0.09188744236925749*x[1]*x[2]*x[3]^3 + 0.021953648369617825*x[2]^2*x[3]^3 + 
  0.00400177260492883*x[3]^4 + 0.011898492118359279*x[1]*x[3]^4 + 
  0.007744527000894741*x[2]*x[3]^4 + 0.0005843383378866362*x[3]^5 + 
  0.8146767174959875*x[4] + 0.28975455696785557*x[1]*x[4] + 
  0.02870678003732808*x[1]^2*x[4] + 0.0001792050754830187*x[1]^3*x[4] + 
  0.004771417940999884*x[1]^4*x[4] + 0.7373487088316814*x[2]*x[4] + 
  0.00756951551759579*x[1]*x[2]*x[4] + 0.20844157437767152*x[1]^2*x[2]*x[4] + 
  0.035889497667893476*x[1]^3*x[2]*x[4] + 0.0011397914097368538*x[2]^2*x[4] + 
  0.05429535455535303*x[1]*x[2]^2*x[4] + 0.010850528739054932*x[1]^2*x[2]^2*x[4] + 
  0.21422307532377288*x[2]^3*x[4] + 0.0010974134653090064*x[1]*x[2]^3*x[4] + 
  0.0004120264368250177*x[2]^4*x[4] + 0.7136028658081554*x[3]*x[4] + 
  0.0017740911590696923*x[1]*x[3]*x[4] + 0.0034193866717547004*x[1]^2*x[3]*x[4] + 
  0.005676894907720873*x[1]^3*x[3]*x[4] + 0.1526072270965341*x[2]*x[3]*x[4] + 
  0.0030605321760627997*x[1]*x[2]*x[3]*x[4] + 0.00009727661777809846*x[1]^2*x[2]*x[3]*
   x[4] + 0.07877665364825194*x[2]^2*x[3]*x[4] + 0.004064620242678005*x[1]*x[2]^2*x[3]*
   x[4] + 0.003335520505960184*x[2]^3*x[3]*x[4] + 0.28861521691284253*x[3]^2*x[4] + 
  0.02941042965636783*x[1]*x[3]^2*x[4] + 0.00021746935970333572*x[1]^2*x[3]^2*x[4] + 
  0.010784789353388361*x[2]*x[3]^2*x[4] + 0.0007486618985693292*x[1]*x[2]*x[3]^2*x[4] + 
  0.009849386658333778*x[2]^2*x[3]^2*x[4] + 0.00042963960446031573*x[3]^3*x[4] + 
  0.07367765680630788*x[1]*x[3]^3*x[4] + 0.01743009838282403*x[2]*x[3]^3*x[4] + 
  0.03288193100830029*x[3]^4*x[4] + 0.24573505879313035*x[4]^2 + 
  0.3429905322474862*x[1]*x[4]^2 + 0.004961860689622251*x[1]^2*x[4]^2 + 
  0.022619983582714484*x[1]^3*x[4]^2 + 0.010726572372997234*x[2]*x[4]^2 + 
  0.10176946688561231*x[1]*x[2]*x[4]^2 + 0.0025381837290582333*x[1]^2*x[2]*x[4]^2 + 
  0.07865257862368143*x[2]^2*x[4]^2 + 0.06182702947499837*x[1]*x[2]^2*x[4]^2 + 
  0.0002534981144690876*x[2]^3*x[4]^2 + 0.0009684100140502721*x[3]*x[4]^2 + 
  0.04963089165149719*x[1]*x[3]*x[4]^2 + 0.010712537887636604*x[1]^2*x[3]*x[4]^2 + 
  0.0034615375884099633*x[2]*x[3]*x[4]^2 + 0.030996636868934592*x[1]*x[2]*x[3]*x[4]^2 + 
  0.0010275592600580796*x[2]^2*x[3]*x[4]^2 + 0.05693048273939116*x[3]^2*x[4]^2 + 
  0.0017737422556637825*x[1]*x[3]^2*x[4]^2 + 0.011046622125664741*x[2]*x[3]^2*x[4]^2 + 
  0.03941176911444959*x[3]^3*x[4]^2 + 0.006766357157425904*x[4]^3 + 
  0.19816628782496395*x[1]*x[4]^3 + 0.05016631260061949*x[1]^2*x[4]^3 + 
  0.30427222927881664*x[2]*x[4]^3 + 0.0022730711136079947*x[1]*x[2]*x[4]^3 + 
  0.049323457273988174*x[2]^2*x[4]^3 + 0.0096034462769309*x[3]*x[4]^3 + 
  0.021788960390696173*x[1]*x[3]*x[4]^3 + 0.0002895566019578634*x[2]*x[3]*x[4]^3 + 
  0.008709111588563854*x[3]^2*x[4]^3 + 0.011441526367366919*x[4]^4 + 
  0.04047096206134594*x[1]*x[4]^4 + 0.00751756447677511*x[2]*x[4]^4 + 
  0.10170052377964965*x[3]*x[4]^4 + 0.00036016325334796564*x[4]^5 + 
  0.17320905685247512*x[5] + 0.059441995714481936*x[1]*x[5] + 
  0.5064556124842657*x[1]^2*x[5] + 0.18293626408670557*x[1]^3*x[5] + 
  0.00029907115792781816*x[1]^4*x[5] + 0.4724737586256838*x[2]*x[5] + 
  0.4911329140298557*x[1]*x[2]*x[5] + 0.00027791038658673157*x[1]^2*x[2]*x[5] + 
  0.026422500036853272*x[1]^3*x[2]*x[5] + 0.008480382217463677*x[2]^2*x[5] + 
  0.0394381870186364*x[1]*x[2]^2*x[5] + 0.000696216659379937*x[1]^2*x[2]^2*x[5] + 
  0.036445248210792675*x[2]^3*x[5] + 0.0007521932555514171*x[1]*x[2]^3*x[5] + 
  0.0011319817676766957*x[2]^4*x[5] + 0.6038398685915572*x[3]*x[5] + 
  0.008332114295915594*x[1]*x[3]*x[5] + 0.054173514223559205*x[1]^2*x[3]*x[5] + 
  0.0007988404581023452*x[1]^3*x[3]*x[5] + 0.05594321614591877*x[2]*x[3]*x[5] + 
  0.022603444196128395*x[1]*x[2]*x[3]*x[5] + 0.026912341174368845*x[1]^2*x[2]*x[3]*
   x[5] + 0.18112702211578538*x[2]^2*x[3]*x[5] + 0.114254892717524*x[1]*x[2]^2*x[3]*
   x[5] + 0.0035266218767229275*x[2]^3*x[3]*x[5] + 0.00821509516311846*x[3]^2*x[5] + 
  0.17595394408211992*x[1]*x[3]^2*x[5] + 0.0328838522492163*x[1]^2*x[3]^2*x[5] + 
  0.03219646690709476*x[2]*x[3]^2*x[5] + 0.0028369599926419036*x[1]*x[2]*x[3]^2*x[5] + 
  0.0035240297631217066*x[2]^2*x[3]^2*x[5] + 0.009030697124486274*x[3]^3*x[5] + 
  0.010463853187394461*x[1]*x[3]^3*x[5] + 0.0018693681690448143*x[2]*x[3]^3*x[5] + 
  0.0037546050318819565*x[3]^4*x[5] + 0.44057441883069265*x[4]*x[5] + 
  0.2768360163222802*x[1]*x[4]*x[5] + 0.06586432037664841*x[1]^2*x[4]*x[5] + 
  0.004049935589165837*x[1]^3*x[4]*x[5] + 0.05972628053866311*x[2]*x[4]*x[5] + 
  0.0010350815923335017*x[1]*x[2]*x[4]*x[5] + 0.00009456483502971165*x[1]^2*x[2]*x[4]*
   x[5] + 0.008194040926532848*x[2]^2*x[4]*x[5] + 0.027198099740178644*x[1]*x[2]^2*x[4]*
   x[5] + 0.0038166040785512423*x[2]^3*x[4]*x[5] + 0.0986872574101289*x[3]*x[4]*x[5] + 
  0.10278963563485552*x[1]*x[3]*x[4]*x[5] + 0.0011449900961500866*x[1]^2*x[3]*x[4]*
   x[5] + 0.0005575778099615266*x[2]*x[3]*x[4]*x[5] + 0.06099438662350444*x[1]*x[2]*x[3]*
   x[4]*x[5] + 0.0007649606697291431*x[2]^2*x[3]*x[4]*x[5] + 
  0.30526250886879924*x[3]^2*x[4]*x[5] + 0.01622366336730543*x[1]*x[3]^2*x[4]*x[5] + 
  0.02562052093899049*x[2]*x[3]^2*x[4]*x[5] + 0.025898875472422915*x[3]^3*x[4]*x[5] + 
  0.06640513257868744*x[4]^2*x[5] + 0.1029522931018822*x[1]*x[4]^2*x[5] + 
  0.00200022802205334*x[1]^2*x[4]^2*x[5] + 0.00037038713949074726*x[2]*x[4]^2*x[5] + 
  0.04035462260432483*x[1]*x[2]*x[4]^2*x[5] + 0.002320219574688007*x[2]^2*x[4]^2*x[5] + 
  0.020076168354424814*x[3]*x[4]^2*x[5] + 0.048427742568908985*x[1]*x[3]*x[4]^2*x[5] + 
  0.000889227511731553*x[2]*x[3]*x[4]^2*x[5] + 0.0002089936918310982*x[3]^2*x[4]^2*
   x[5] + 0.010876045348786948*x[4]^3*x[5] + 0.011477202201494962*x[1]*x[4]^3*x[5] + 
  0.18094234092028275*x[2]*x[4]^3*x[5] + 0.0007689772229517157*x[3]*x[4]^3*x[5] + 
  0.035389001167262936*x[4]^4*x[5] + 0.04072818927060682*x[5]^2 + 
  0.13660896440294829*x[1]*x[5]^2 + 0.43801466038080966*x[1]^2*x[5]^2 + 
  0.0007757905637675677*x[1]^3*x[5]^2 + 0.3130060157127575*x[2]*x[5]^2 + 
  0.0025932089872475928*x[1]*x[2]*x[5]^2 + 0.1187819142994332*x[1]^2*x[2]*x[5]^2 + 
  0.14716913386198113*x[2]^2*x[5]^2 + 0.00034307204678808397*x[1]*x[2]^2*x[5]^2 + 
  0.01652177621412203*x[2]^3*x[5]^2 + 0.02193323478500852*x[3]*x[5]^2 + 
  0.005520786515221187*x[1]*x[3]*x[5]^2 + 0.002849834721120786*x[1]^2*x[3]*x[5]^2 + 
  0.23221088288816652*x[2]*x[3]*x[5]^2 + 0.0003304380493867622*x[1]*x[2]*x[3]*x[5]^2 + 
  0.00999186423728579*x[2]^2*x[3]*x[5]^2 + 0.027712488183944957*x[3]^2*x[5]^2 + 
  0.188386895337007*x[1]*x[3]^2*x[5]^2 + 0.16945877726914552*x[2]*x[3]^2*x[5]^2 + 
  0.00034190705098783404*x[3]^3*x[5]^2 + 0.09610605450175996*x[4]*x[5]^2 + 
  0.0017873213502568905*x[1]*x[4]*x[5]^2 + 0.05631962618795378*x[1]^2*x[4]*x[5]^2 + 
  0.2953040066824302*x[2]*x[4]*x[5]^2 + 0.03669229748633779*x[1]*x[2]*x[4]*x[5]^2 + 
  0.10368210805468676*x[2]^2*x[4]*x[5]^2 + 0.04272654179086045*x[3]*x[4]*x[5]^2 + 
  0.013913397274971043*x[1]*x[3]*x[4]*x[5]^2 + 0.15601124268691974*x[2]*x[3]*x[4]*
   x[5]^2 + 0.014884658544949228*x[3]^2*x[4]*x[5]^2 + 
  0.0003106486751821634*x[4]^2*x[5]^2 + 0.004548828001713855*x[1]*x[4]^2*x[5]^2 + 
  0.09880582401044023*x[2]*x[4]^2*x[5]^2 + 0.00853772668582695*x[3]*x[4]^2*x[5]^2 + 
  0.007605925385335723*x[4]^3*x[5]^2 + 0.31960267330973036*x[5]^3 + 
  0.006365828199157354*x[1]*x[5]^3 + 0.18551373412310418*x[1]^2*x[5]^3 + 
  0.013244300299405614*x[2]*x[5]^3 + 0.017145280282651488*x[1]*x[2]*x[5]^3 + 
  0.005073535046056006*x[2]^2*x[5]^3 + 0.09010364330124006*x[3]*x[5]^3 + 
  0.005035255530430199*x[1]*x[3]*x[5]^3 + 0.032672159425846635*x[2]*x[3]*x[5]^3 + 
  0.012580338525233952*x[3]^2*x[5]^3 + 0.03163795878382286*x[4]*x[5]^3 + 
  0.047942597447498236*x[1]*x[4]*x[5]^3 + 0.0000854584314823532*x[2]*x[4]*x[5]^3 + 
  0.02592605876437933*x[3]*x[4]*x[5]^3 + 0.019279497584226566*x[4]^2*x[5]^3 + 
  0.06143974985761201*x[5]^4 + 0.006460585265824627*x[1]*x[5]^4 + 
  0.2227312331991271*x[2]*x[5]^4 + 0.0006765287915386199*x[3]*x[5]^4 + 
  0.00004805050930237017*x[4]*x[5]^4 + 0.027413694740774566*x[5]^5),
    x[1]^2+x[2]^2+x[3]^2+x[4]^2-1+
    0.00012405266272129466*(1 + 0.5100879465985098*x[1] + 0.13674387509106709*x[1]^2 + 
  0.9395135349741788*x[2] + 0.05835079480502024*x[1]*x[2] + 0.6085258295366837*x[2]^2 + 
  0.568006784133888*x[3] + 0.3596845533010325*x[1]*x[3] + 
  0.12205655999996853*x[2]*x[3] + 0.7571804248419666*x[3]^2 + 0.3230336322898173*x[4] + 
  0.11685038978697308*x[1]*x[4] + 0.039155250613630226*x[2]*x[4] + 
  0.03056273763799633*x[3]*x[4] + 0.3242707581564692*x[4]^2 + 0.5891736574466393*x[5] + 
  0.20265444154013473*x[1]*x[5] + 0.009830737397804717*x[2]*x[5] + 
  0.11063938983156173*x[3]*x[5] + 0.42557876488770907*x[4]*x[5] + 
  0.3255943853631123*x[5]^2)
]

d=length(F) # codimension of variety
k = n-d # dimension of variety
@polyvar lambda[1:d] mu[1:d]; # lagrange multipliers

In [159]:
# Compute the bottlenecks

grad = differentiate(F, x)
G = subs(F, x => y)
grady = subs(grad, x => y)

system = [F; G; map(j -> x[j]-y[j]-dot(lambda, grad[:, j]), 1:n); map(j -> x[j]-y[j]-dot(mu, grady[:, j]), 1:n)]
result = solve(system, start_system = :polyhedral)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /Users/daniel/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Computing mixed cells... 5091 	 Time: 0:00:01
  mixed_volume:  214639

LoadError: OverflowError: Cannot compute a start system.

In [138]:
# Choose the size of the grid as the smallest bottleneck

bottlenecks = map(s -> (s[1:n], s[n+1:2*n]), real_solutions(nonsingular(result)))
bn_lengths = sort!(map(b -> (norm(b[1]-b[2]), b), bottlenecks), by = a -> a[1])
δ = bn_lengths[1][1]/3 # grid size

0.6662883707165751

In [139]:
# Compute the bounding box by computing the EDD starting from the center of the largest bottleneck

q = bn_lengths[end][2][1] + (bn_lengths[end][2][2]-bn_lengths[end][2][1])/2 + randn(Float64, n)*10^(-10)
system = [F; map(j -> x[j]-q[j]-dot(lambda, grad[:, j]), 1:n)]
result = solve(system, start_system = :polyhedral)

Result{Vector{ComplexF64}} with 144 solutions
• 66 non-singular solutions (10 real)
• 78 singular solutions (0 real)
• 144 paths tracked
• random seed: 964736
• multiplicity table of singular solutions:
┌───────┬───────┬────────┬────────────┐
│ mult. │ total │ # real │ # non-real │
├───────┼───────┼────────┼────────────┤
│   1   │  78   │   0    │     78     │
└───────┴───────┴────────┴────────────┘


In [140]:
# Extract farthest point from q to X and use as box length

critical_points = sort!(map(c -> (norm(c[1:n]-q), c[1:n]), real_solutions(nonsingular(result))), by = a -> a[1])
b = critical_points[end][1]
indices = [i for i in -b:δ:b];

In [171]:
# guess:
# smallest bottleneck size
# box length
q = randn(Float64, n)*10^(-10)
δ = 0.3
b = 1.1
indices = [i for i in -b:δ:b];

In [172]:
# Compute basic sample

samples = []
counter = 0

start_time = time_ns()
for s in IterTools.subsets(1:n, k)
    Ft = [F; map(i -> x[s[i]]-p[i]-q[s[i]], 1:k)]
    p₀ = randn(ComplexF64, k)
    F_p₀ = subs(Ft, p[1:k] => p₀)
    result_p₀ = solve(F_p₀)
    S_p₀ = solutions(result_p₀)
    
    # Construct the PathTracker
    tracker = HomotopyContinuation.pathtracker(Ft; parameters=p[1:k], generic_parameters=p₀)
    for p1 in Iterators.product(map(j-> 1:length(indices), s)...)
        counter += length(S_p₀)
        for s1 in S_p₀
            result = track(tracker, s1; target_parameters=map(j -> indices[p1[j]], 1:k))
            # check that the tracking was successfull
            if is_success(result) && is_real(result)
                push!(samples, real(solution(result)))
            end
        end
    end
end
time_basic_sample = time_ns()-start_time;

In [173]:
# Compute extra sample

extra_samples = []
extra_counter = 0

start_time = time_ns()
for l in 1:k-1
    for s in IterTools.subsets(1:n, l)
        Ft = [F; map(i -> x[s[i]]-p[i]-q[s[i]], 1:l)] 
        grad = differentiate(Ft, x)
        system = [Ft; map(j -> x[j]-y[j]-dot(gamma[1:n-k+l], grad[:, j]), 1:n)]
        
        p₀ = randn(ComplexF64, n+l)
        F_p₀ = subs(system, [y; p[1:l]] => p₀)
        result_p₀ = solve(F_p₀)
        S_p₀ = solutions(result_p₀)

        # Construct the PathTracker
        tracker = HomotopyContinuation.pathtracker(system; parameters=[y; p[1:l]], generic_parameters=p₀)
        for p1 in Iterators.product(map(j-> 1:length(indices), s)...)
            extra_counter += length(S_p₀)
            for s1 in S_p₀
                result = track(tracker, s1; target_parameters=[randn(Float64, n); map(j -> indices[p1[j]], 1:l)])
                # check that the tracking was successfull
                if is_success(result) && is_real(result)
                    push!(extra_samples, real(solution(result))[1:n])
                end
            end
        end
    end
end
time_extra_sample = time_ns()-start_time;

Tracking 31250 paths... 100%|████████████████████████████████████████| Time: 0:00:40
  # paths tracked:                  31250
  # non-singular solutions (real):  50 (0)
  # singular solutions (real):      1 (0)
  # total solutions (real):         51 (0)


In [174]:
# Summary

println("grid size: ", δ)
println("length of bounding box: ", b)
println("number of tracked paths: ", counter+extra_counter)
println("total time: ", (time_basic_sample+time_extra_sample)*10^(-9), " s")
println("|E_δ|: " , length(samples), ", |E'_δ|: ", length(extra_samples))
println("total number of samples: ", length(samples)+length(extra_samples))

grid size: 0.3
length of bounding box: 1.1
number of tracked paths: 96936
total time: 631.0817843330003 s
|E_δ|: 2794, |E'_δ|: 1365
total number of samples: 4159


# Plotting the samples

In [175]:
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [176]:
# Plot the sampled points

S = reduce(hcat, vcat(samples, extra_samples))
if n < 3
    scatter(S[1,:], S[2,:])
else
    scatter(S[1,:], S[2,:], S[3,:])
end

<!DOCTYPE html>
 
 
 Plots.jl

# Homology

In [177]:
using Eirene

In [178]:
#Compute distance matrix

ϵ = 2*δ
D = zeros((length(samples), length(samples))) #Initialize distance matrix

neighbour_lists = []
for i in 1:length(samples)
    push!(neighbour_lists, [])
end
candidate_lists = []

for i in 1:length(samples)
    candidate_list = []
    for j in (i+1):length(samples)
        dist = norm(samples[i]-samples[j])
        if dist < sqrt(8)*δ
            if dist < ϵ
                push!(neighbour_lists[i], j)
                push!(neighbour_lists[j], i)
            else
                push!(candidate_list, j)
            end
        end
        D[i, j] = dist
        D[j, i] = dist
    end
    push!(candidate_lists, candidate_list)
end

In [179]:
# Modify distance matrix to add edges in VR-complex

DM = deepcopy(D) #Modified distance matrix

thresh = 2*δ - 10^(-10)
for i in 1:length(samples)
    for j in candidate_lists[i]
        for k in neighbour_lists[j]
            if D[i, k] < ϵ && D[j, k] < ϵ
                DM[i, j] = thresh
                DM[j, i] = thresh
                break
            end
        end
    end
end

In [180]:
# Homology computation using Eirene using VR-complex with distance ϵ
C = eirene(DM, model="vr", maxdim=1, minrad=ϵ, maxrad=ϵ);

In [181]:
println("0-th Betti number: ", betticurve(C, dim=0)[1, 2])
println("1-th Betti number: ", betticurve(C, dim=1)[1, 2])

0-th Betti number: 1.0
1-th Betti number: 0.0
